# Algoritmo para Inferir red booleana

*Para inferir la red booleana se creara un funcion que reciba como parametros la funcion de busqueda, la funcion criterio y la matriz de expresion genica*

## Importacion de Librerias

In [228]:
import numpy as np
from typing import Callable, List, Tuple
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import os
from collections import Counter
from math import log2

from typing import List, Callable
from pyeasyga import pyeasyga
from auxAG import _inicializar_poblacion, _evaluar_fitness, _seleccionar_padres, _cruzar_poblacion, _mutar_poblacion
from claseAG import structureAG

## Tipo de Funciones para inferir una red booleana

### Funciones de Busqueda y Evaluacion

### funcion(es) de busqueda

In [229]:
# --- Funciones de Búsqueda (Ejemplos) ---

def busqueda_exhaustiva(target_gene_index: int, matriz_expresion: np.ndarray, funcion_criterio: Callable) -> List[int]:
    """
    Busca exhaustivamente los mejores predictores para un gen objetivo, considerando subconjuntos de 1, 2 y 3 genes.

    Args:
        target_gene_index (int): El índice del gen objetivo.
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        funcion_criterio (Callable): La función de criterio para evaluar los predictores.

    Returns:
        List[int]: Una lista con los índices de los mejores predictores encontrados.
    """
    print(f"\nSearching predictors for target gene {target_gene_index}")
    num_genes = matriz_expresion.shape[1]
    mejor_predictores = []
    mejor_criterio = float('inf')  # Inicializamos con un valor alto

    # Iterar sobre los subconjuntos de predictores de tamaño 1, 2 y 3
    for size in range(1, min(4, num_genes)):
      for predictores_indices in _combinations_no_repetir(list(range(num_genes)), size, target_gene_index):
        
        criterio = funcion_criterio(matriz_expresion, target_gene_index, predictores_indices)

        if criterio < mejor_criterio:
            mejor_criterio = criterio
            mejor_predictores = predictores_indices
    for size in range(1, min(4, num_genes)):
        print(f"\nTesting predictor sets of size {size}")
        # ...rest of loop code...

    return mejor_predictores

def _combinations_no_repetir(lista: List, n: int, target_gene_index: int) -> List:
    """
    Genera combinaciones de n elementos de una lista, excluyendo un índice específico.
    """
    if n==1:
        for i in lista:
            if i != target_gene_index:
                yield [i]
        return

    for i in range(len(lista)):
        if lista[i] != target_gene_index:
           for sub_comb in _combinations_no_repetir(lista[i+1:], n - 1, target_gene_index):
              yield [lista[i]]+sub_comb

def busqueda_sfs(target_gene_index: int, matriz_expresion: np.ndarray, funcion_criterio: Callable) -> List[int]:
    """
    Realiza búsqueda secuencial hacia adelante (SFS) mejorada.

    Args:
        target_gene_index (int): Índice del gen objetivo
        matriz_expresion (np.ndarray): Matriz de expresión génica
        funcion_criterio (Callable): Función para evaluar predictores

    Returns:
        List[int]: Lista de índices de mejores predictores
    """
    print(f"\nBuscando predictores con SFS para gen objetivo {target_gene_index}")
    
    # Inicialización
    num_genes = matriz_expresion.shape[1]
    selected_predictors = set()  # Usar set para predictores seleccionados
    available_genes = set(range(num_genes)) - {target_gene_index}
    max_predictors = min(3, num_genes - 1)
    
    # Criterio de parada temprana
    best_score = float('-inf')
    no_improvement_count = 0
    max_no_improvement = 2
    
    while len(selected_predictors) < max_predictors and no_improvement_count < max_no_improvement:
        best_new_predictor = None
        best_new_score = float('-inf')
        
        # Evaluar candidatos
        for candidate in available_genes:
            current_predictors = list(selected_predictors | {candidate})
            score = funcion_criterio(matriz_expresion, target_gene_index, current_predictors)
            
            if score > best_new_score:
                best_new_score = score
                best_new_predictor = candidate
                
        # Verificar si hay mejora
        if best_new_score > best_score:
            best_score = best_new_score
            selected_predictors.add(best_new_predictor)
            available_genes.remove(best_new_predictor)
            no_improvement_count = 0
            print(f"Añadido predictor {best_new_predictor} con score {best_new_score:.4f}")
        else:
            no_improvement_count += 1
            print(f"No hay mejora en iteración {len(selected_predictors) + 1}")
            
    result = list(selected_predictors)
    print(f"\nPredictores finales para gen {target_gene_index}: {result}")
    print(f"Score final: {best_score:.4f}")
    
    return result

### funcion(es) de penalizacion

In [230]:
# --- Funciones de Criterio (Ejemplos) ---

def error_absoluto(matriz_expresion: np.ndarray, target_gene_index: int, predictores_indices: List[int]) -> float:
    """
    Calcula el error absoluto promedio entre la expresión del gen objetivo y la predicción booleana basada en los predictores.
    
    Args:
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        target_gene_index (int): El índice del gen objetivo.
        predictores_indices (List[int]): La lista de índices de los genes predictores.

    Returns:
        float: El error absoluto promedio.
    """
    target_gene = matriz_expresion[:, target_gene_index]
    num_condiciones = len(target_gene)

    if not predictores_indices:
        # En el caso de que no existan predictores devolvemos un valor alto
        return float('inf')

    predicciones = np.zeros(num_condiciones)
    for condicion in range(num_condiciones):
        valores_predictores = [matriz_expresion[condicion, i] for i in predictores_indices] 

        # Implementacion del OR
        prediccion = 0
        if any([True if value > 0 else False for value in valores_predictores]):
          prediccion = 1

        predicciones[condicion] = prediccion


    return np.mean(np.abs(target_gene - predicciones))
def _calcular_entropia(p: float) -> float:
    """Calcula la entropía para una probabilidad dada."""
    if p == 0 or p == 1:
        return 0
    return -p * log2(p) - (1 - p) * log2(1 - p)

def _calcular_entropia_gen(gen: np.ndarray) -> float:
    """Calcula la entropía para un gen dado."""
    num_condiciones = len(gen)
    p = np.sum(gen) / num_condiciones
    return _calcular_entropia(p)

def informacion_mutua_condicional(matriz_expresion: np.ndarray, target_gene_index: int, predictores_indices: List[int]) -> float:
    """
    Calcula la información mutua condicionada entre el gen objetivo y los predictores.
    Esta implementacion considera el OR como funcion booleana.

    Args:
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        target_gene_index (int): El índice del gen objetivo.
        predictores_indices (List[int]): La lista de índices de los genes predictores.

    Returns:
        float: La información mutua condicionada.
    """
    target_gene = matriz_expresion[:, target_gene_index]
    num_condiciones = len(target_gene)

    if not predictores_indices:
         # En el caso de que no existan predictores devolvemos un valor bajo
        return float('-inf')
    
    entropia_target = _calcular_entropia_gen(target_gene)
    
    # Calcular la entropía condicional
    entropia_condicional = 0
    for predictores_estado in [0, 1]:
        indices = []
        for condicion in range(num_condiciones):
            valores_predictores = [matriz_expresion[condicion, i] for i in predictores_indices] 
            
            # Implementacion del OR
            prediccion = 0
            if any([True if value > 0 else False for value in valores_predictores]):
               prediccion = 1
            
            if prediccion == predictores_estado:
                indices.append(condicion)
                
        if len(indices) > 0:
            entropia_condicional += (len(indices) / num_condiciones) * _calcular_entropia_gen(target_gene[indices])
    
    return entropia_target - entropia_condicional

def informacion_mutua_penalizada(matriz_expresion: np.ndarray, target_gene_index: int, predictores_indices: List[int]) -> float:
    """
    Calcula la información mutua penalizada (IMP) entre el gen objetivo y los predictores.
    
    Args:
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        target_gene_index (int): El índice del gen objetivo.
        predictores_indices (List[int]): La lista de índices de los genes predictores.
    
    Returns:
        float: El valor de la información mutua penalizada.
    """
    target_gene = matriz_expresion[:, target_gene_index]
    num_condiciones = len(target_gene)
    
    if not predictores_indices:
        # En el caso de que no existan predictores devolvemos un valor bajo
        return float('-inf')
    
    # Obtener las combinaciones posibles de los predictores
    predictores = matriz_expresion[:, predictores_indices]
    
    # Concatenar los predictores para formar estados combinados
    if predictores.ndim == 1:
        estados_predictores = predictores
    else:
        # Convertir combinaciones binarias a índices únicos
        potencias = 2 ** np.arange(len(predictores_indices))
        estados_predictores = np.dot(predictores, potencias)
    
    # Calcular frecuencias conjuntas
    conjunto_estados = np.vstack((estados_predictores, target_gene)).T
    conteo_conjunto = Counter(map(tuple, conjunto_estados))
    
    # Calcular frecuencias marginales
    conteo_predictores = Counter(estados_predictores)
    conteo_target = Counter(target_gene)
    
    # Calcular el número total de estados posibles
    num_estados_predictores = 2 ** len(predictores_indices)
    num_estados_target = 2  # Binario
    num_estados_posibles = num_estados_predictores * num_estados_target
    
    # Penalización alfa
    alfa = 1 / num_estados_posibles
    
    # Calcular la información mutua penalizada
    imp = 0.0
    total_instancias = num_condiciones + alfa * num_estados_posibles  # Ajuste por penalización
    
    for estado_pred in range(num_estados_predictores):
        for estado_target in [0, 1]:
            conteo_xy = conteo_conjunto.get((estado_pred, estado_target), 0)
            p_xy = (conteo_xy + alfa) / total_instancias
            
            conteo_x = conteo_predictores.get(estado_pred, 0)
            p_x = (conteo_x + alfa * num_estados_target) / total_instancias
            
            conteo_y = conteo_target.get(estado_target, 0)
            p_y = (conteo_y + alfa * num_estados_predictores) / total_instancias
            
            if p_xy > 0 and p_x > 0 and p_y > 0:
                imp += p_xy * np.log2(p_xy / (p_x * p_y))
    
    return imp

## Busqueda de seleccion de caracsteristicas

In [231]:
# --- Función de Búsqueda de Algoritmo Genético ---
def busqueda_algoritmo_genetico(
    target_gene_index: int,
    matriz_expresion: np.ndarray,
    funcion_criterio: Callable,
    tamano_poblacion: int = 100,
    num_generaciones: int = 100,
    prob_mutacion: float = 0.1,
    prob_cruce: float = 0.7
) -> List[int]:
    """
    Realiza la búsqueda de predictores usando un algoritmo genético.

    Args:
        target_gene_index (int): El índice del gen objetivo.
        matriz_expresion (np.ndarray): La matriz de expresión génica.
        funcion_criterio (Callable): La función de criterio para evaluar los predictores.
        tamano_poblacion (int): El tamaño de la población.
        num_generaciones (int): El número de generaciones del AG.
        prob_mutacion (float): Probabilidad de mutación.
        prob_cruce (float): Probabilidad de cruce.
    Returns:
        List[int]: Una lista con los índices de los mejores predictores encontrados.
    """
    print(f"\nSearching predictors with Genetic Algorithm for target gene {target_gene_index}")
    num_genes = matriz_expresion.shape[1]
    
    # -- El problema considera la selección de predictores en función al número
    #    de genes del problema.
    #    Sólo estos índices serán sometidos a los procesos de selección, cruce y
    #    mutación.
    #    La solución final será el conjunto de predictores que mejor se ajusten
    #    a la función de criterio
    problema =  list(range(num_genes))
    problema.remove(target_gene_index)
    
    ga = claseAG.structureAG(problema, nroIndividuos = tamano_poblacion)

    # -- Ejecutar el algoritmo genético
    mejor_individuo = ga.Ejecutar(matriz_expresion, target_gene_index, funcion_criterio)
    if mejor_individuo is not None:
      
      return [problema[i] for i in mejor_individuo[1]]
    else:
      return []

### Funciones de Lectura

In [232]:
# --- Funciones de Lectura de Archivo ---

def leer_matriz_deg(archivo_path: str) -> np.ndarray:
    """
   Lee una matriz de expresión de un archivo .deg, donde la primera línea tiene el número de tiempos y el número de genes

    Args:
      archivo_path: La ruta al archivo .deg.

    Returns:
        np.ndarray: La matriz de expresión como un array NumPy.
    """

    if not os.path.exists(archivo_path):
        raise FileNotFoundError(f"El archivo '{archivo_path}' no fue encontrado.")

    try:
        with open(archivo_path, "r") as file:
          lineas = file.readlines()
          if not lineas:
            raise ValueError("El archivo esta vacio.")
          
          header = lineas[0].strip().split()
          if len(header) != 2:
              raise ValueError("La primera línea debe tener dos valores (número de tiempos y número de genes).")
          try:
              num_tiempos = int(header[0])
              num_genes = int(header[1])
          except ValueError:
              raise ValueError("Los valores en la primera línea deben ser números enteros.")

          if num_tiempos <= 0 or num_genes <= 0:
                raise ValueError("Los valores de número de tiempos y número de genes deben ser mayores a 0.")

          matriz = []
          for linea in lineas[1:]:
             linea = linea.strip().split()
             try:
               fila = [int(x) for x in linea]
               if len(fila) != num_genes:
                    raise ValueError(f"La fila {linea} tiene un número incorrecto de columnas.")
               matriz.append(fila)
             except ValueError:
                raise ValueError(f"Error en la conversión de datos en la línea: '{linea}'. Asegúrese de que todos los valores sean enteros.")

          if len(matriz) != num_tiempos:
              raise ValueError(f"El número de filas {len(matriz)} no coincide con el número de instantes de tiempo {num_tiempos}.")
          return np.array(matriz)
    except Exception as e:
       raise Exception(f"Error al leer el archivo {archivo_path}: {e}")


#Lectura .ged
def leer_matriz_ged(archivo_path: str) -> np.ndarray:
    """
    Lee un archivo .ged y devuelve una matriz de expresión génica.

    Args:
        archivo_path (str): Ruta al archivo .ged

    Returns:
        np.ndarray: Matriz de expresión génica (tiempos x genes)
    """
    if not os.path.exists(archivo_path):
        raise FileNotFoundError(f"El archivo '{archivo_path}' no fue encontrado.")

    try:
        with open(archivo_path, 'r') as file:
            # Leer la primera línea y extraer los valores necesarios
            primera_linea = file.readline().strip()
            valores_iniciales = primera_linea.split()
            
            if len(valores_iniciales) < 2:
                raise ValueError("La primera línea debe tener al menos dos valores (número de tiempos y número de genes).")
            
            num_tiempos = int(valores_iniciales[0])
            num_genes = int(valores_iniciales[1])
            # Ignorar el valor extra (-1.0) en la primera línea
            
            # Leer el resto del archivo y construir la matriz
            datos = []
            for linea in file:
                linea = linea.strip()
                if linea:  # Si la línea no está vacía
                    valores = [int(float(valor)) for valor in linea.split()]
                    if len(valores) != num_genes:
                        raise ValueError(f"La línea tiene un número incorrecto de columnas. Se esperaban {num_genes}, pero se encontraron {len(valores)}.")
                    datos.append(valores)
            
            matriz = np.array(datos)
            
            # Verificar que la matriz tenga las dimensiones correctas
            if matriz.shape != (num_tiempos, num_genes):
                raise ValueError(f"La matriz de expresión tiene dimensiones {matriz.shape}, pero se esperaban ({num_tiempos}, {num_genes}).")
            
            return matriz
    except Exception as e:
        raise Exception(f"Error al leer el archivo {archivo_path}: {e}")
    
def leer_red_ylu(archivo_path: str, num_genes: int) -> dict:
    """
    Lee la red desde un archivo .ylu modificando para ignorar valores extra.

    Args:
        archivo_path (str): Ruta al archivo .ylu
        num_genes (int): Número de genes en la red

    Returns:
        dict: Diccionario con las relaciones gen->predictores
    """
    if not os.path.exists(archivo_path):
        raise FileNotFoundError(f"El archivo '{archivo_path}' no fue encontrado.")

    red = {}
    try:
        with open(archivo_path, "r") as file:
            lines = file.readlines()
            for line in lines:
                if not line.strip() or line.startswith('RANDOM') or line.startswith('NN'):
                    continue

                if '|' in line:
                    parts = line.strip().split('|')
                    if len(parts) >= 2:
                        predictors = parts[0].strip().split(',')
                        target = parts[1].strip()
                        # Ignorar valores adicionales después del target

                        try:
                            target_gene = int(target)
                            # Validar índice del gen objetivo
                            if target_gene >= num_genes:
                                continue

                            predictor_genes = []
                            for p in predictors:
                                if p.strip().isdigit():
                                    pred = int(p)
                                    # Validar índice del predictor
                                    if pred < num_genes:
                                        predictor_genes.append(pred)
                            red[target_gene] = predictor_genes
                        except ValueError:
                            continue

        return red
    except Exception as e:
        raise Exception(f"Error al leer el archivo {archivo_path}: {e}")


## Clase para validar una red booleana.

In [233]:
# --- Módulo de Evaluación ---
class funcion_evaluacion:

    def __init__(self, num_genes: int):
        self.num_genes = num_genes
        
    def comparar_redes_linea_a_linea(self, red_inferida: dict, red_verdadera: dict) -> np.ndarray:
        """Compara la red inferida con la red verdadera línea por línea."""
        y_true = []
        y_pred = []
        for gen_index in range(self.num_genes):
          if gen_index in red_verdadera:
            true_predictors = set(red_verdadera.get(gen_index, []))
            pred_predictors = set(red_inferida.get(gen_index, []))

            tp = len(true_predictors.intersection(pred_predictors))
            fp = len(pred_predictors - true_predictors)
            fn = len(true_predictors - pred_predictors)
            
            # Append values for confusion matrix calculation
            y_true.append(str((tp,fp,fn)))
            y_pred.append(str((tp,fp,fn)))

        labels = sorted(list(set(y_true)))
        # Convert to numeric index for confusion matrix
        y_true_idx = [labels.index(label) for label in y_true]
        y_pred_idx = [labels.index(label) for label in y_pred]
        
        conf_matrix = confusion_matrix(y_true_idx, y_pred_idx, labels = range(len(labels)))
        return conf_matrix, labels

    def generar_matriz_confusion_con_etiquetas(self, red_verdadera: dict, target_names: List[str]) -> None:
        """Genera la matriz de confusión y la visualiza con etiquetas."""
                
        num_genes = self.matriz_expresion.shape[1]
        
        # Crear matrices de adyacencia
        matriz_verdadera = np.zeros((num_genes, num_genes))
        for target, predictors in red_verdadera.items():
            for predictor in predictors:
                matriz_verdadera[predictor, target] = 1

        matriz_inferida = np.zeros((num_genes, num_genes))
        for target, predictors in self.red.items():
            for predictor in predictors:
                matriz_inferida[predictor, target] = 1

        # Aplanar las matrices
        y_true = matriz_verdadera.flatten()
        y_pred = matriz_inferida.flatten()

        # **Añadir verificación de etiquetas únicas**
        unique_labels = np.unique(np.concatenate((y_true, y_pred)))
        if len(unique_labels) < 2:
            print("No se puede generar la matriz de confusión con etiquetas porque y_true o y_pred contienen una sola clase.")
            return
        else:
            cm = confusion_matrix(y_true, y_pred, labels=unique_labels)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=unique_labels)
            disp.plot()
            plt.show()
    def evaluar_precision(self, red_inferida: dict, red_verdadera: dict) -> float:
        """
        Evalúa la precisión de la red inferida comparándola con la red verdadera.
        
        Args:
           red_verdadera (dict): Diccionario con la red verdadera
        
        Returns:
            float: Precisión del modelo
        """
        matriz_inferida = np.zeros((self.num_genes, self.num_genes))
        matriz_verdadera = np.zeros((self.num_genes, self.num_genes))

        # Convertir red inferida a matriz de adyacencia
        for gen, predictores in red_inferida.items():
          for predictor in predictores:
            matriz_inferida[gen][predictor] = 1

        # Convertir red verdadera a matriz de adyacencia
        for gen, predictores in red_verdadera.items():
          for predictor in predictores:
            matriz_verdadera[gen][predictor] = 1

        # Aplanar las matrices para usar confusion_matrix
        y_true = matriz_verdadera.flatten()
        y_pred = matriz_inferida.flatten()
        
        # Calcular matriz de confusión
        cm = confusion_matrix(y_true, y_pred)

        # Calcular las metricas
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, zero_division = 0)
        recall = recall_score(y_true, y_pred, zero_division = 0)
        f1 = f1_score(y_true, y_pred, zero_division = 0)
        
        print("\nMatriz de Confusión (Precisión):")
        print(cm)
        print(f"\nPrecisión: {precision:.4f}")
        print(f"Exhaustividad (Recall): {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print(f"Accuracy: {accuracy:.4f}")

        return accuracy


## Clase para inferir una red booleana

In [234]:


class InferirRedBooleana:
    def __init__(self, funcion_busqueda: Callable, funcion_criterio: Callable, matriz_expresion: np.ndarray):
        """
        Inicializa la clase InferirRedBooleana.

        Args:
            funcion_busqueda (Callable): Una función que busca los mejores predictores.
            funcion_criterio (Callable): Una función que evalúa la calidad de un conjunto de predictores.
            matriz_expresion (np.ndarray): La matriz de expresión génica (genes x condiciones).
        """
        self.funcion_busqueda = funcion_busqueda
        self.funcion_criterio = funcion_criterio
        self.matriz_expresion = matriz_expresion
        self.num_genes = matriz_expresion.shape[1]
        self.red = {}  # Diccionario para almacenar la red inferida: {gen: [predictores]}
        self.funcion_evaluacion = funcion_evaluacion(self.num_genes)

    def inferir_red(self) -> dict:
        """
        Infiere la red de regulación génica utilizando las funciones proporcionadas.

        Returns:
            dict: Un diccionario que representa la red inferida (gen: lista_de_predictores).
        """
        for target_gene_index in range(self.num_genes):
            predictores = self.funcion_busqueda(target_gene_index, self.matriz_expresion, self.funcion_criterio)
            self.red[target_gene_index] = predictores
        return self.red

    def obtener_red(self) -> dict:
        """
        Retorna la red inferida.

        Returns:
            dict:  Un diccionario que representa la red inferida (gen: lista_de_predictores).
        """
        return self.red
    
    def generar_matriz_confusion_con_etiquetas(self, red_verdadera: dict, target_names: List[str]) -> None:
      self.funcion_evaluacion.generar_matriz_confusion_con_etiquetas(self.red, red_verdadera, target_names ,self.matriz_expresion)
    
    def evaluar_precision(self, red_verdadera: dict) -> float:
        return self.funcion_evaluacion.evaluar_precision(self.red,red_verdadera)

    

## Salidas

In [235]:
def guardar_red_inferida(red_inferida: dict, funcion_busqueda: str, funcion_criterio: str) -> None:
    """
    Guarda la red inferida en un archivo JSON.
    
    Args:
        red_inferida (dict): Red inferida
        nombre_archivo (str, optional): Nombre personalizado del archivo
    """
    import os
    import json
    from datetime import datetime
    
    # Crear directorio si no existe
    directorio = 'redes_inferidas'
    if not os.path.exists(directorio):
        os.makedirs(directorio)
        
    # Generar ID único con timestamp
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Crear nombre de archivo con formato específico
    nombre_archivo = f'red_inferida_{funcion_busqueda}_{funcion_criterio}_{timestamp}.json'
    ruta_archivo = os.path.join(directorio, nombre_archivo)
    
    # Preparar datos para guardar
    datos = {
        'metadata': {
            'id': timestamp,
            'funcion_busqueda': funcion_busqueda,
            'funcion_criterio': funcion_criterio,
            'fecha_creacion': datetime.now().isoformat()
        },
        'red_inferida': red_inferida
    }
    
    # Guardar en formato JSON
    with open(ruta_archivo, 'w') as f:
        json.dump(datos, f, indent=4)
        
    print(f"\nRed inferida guardada en: {ruta_archivo}")




## Modulo de Usabilidad

In [236]:
# --- Ejemplo de uso ---

# Modificar código principal:
if __name__ == '__main__':
    try:
        # Leer matriz de expresión
        matriz_exp = leer_matriz_ged(r"C:\Users\josep\OneDrive\Documentos\Semestre 2024-II\BioInformatica\Bioinfo_ DS\deg_999.ged")
        num_genes = matriz_exp.shape[1]
        
        # Leer red verdadera con límite de genes
        red_verdadera = leer_red_ylu(r"C:\Users\josep\OneDrive\Documentos\Semestre 2024-II\BioInformatica\Bioinfo_ DS\red_999.ylu", num_genes)
        
        # Leer red verdadera con límite de genes
        
        # Define las opciones disponibles para el usuario
        opciones_busqueda = {
            "1": ("busqueda_exhaustiva", busqueda_exhaustiva),
            "2": ("busqueda_sfs", busqueda_sfs),
            "3": ("busqueda_algoritmo_genetico", busqueda_algoritmo_genetico),
        }

        opciones_criterio = {
            "1": ("error_absoluto", error_absoluto),
            "2": ("informacion_mutua_condicional", informacion_mutua_condicional),
            "3": ("informacion_mutua_penalizada", informacion_mutua_penalizada),
        }
        
        while True:
           
            print("\n--- MENÚ DE OPCIONES ---")
            print("Funciones de Búsqueda:")
            for key, (nombre, _) in opciones_busqueda.items():
                print(f"{key}. {nombre}")

            print("\nFunciones de Criterio:")
            for key, (nombre, _) in opciones_criterio.items():
                print(f"{key}. {nombre}")

            opcion_busqueda = input("\nSeleccione el número de la función de búsqueda: ")
            opcion_criterio = input("Seleccione el número de la función de criterio: ")

            if opcion_busqueda in opciones_busqueda and opcion_criterio in opciones_criterio:
                nombre_busqueda, funcion_busqueda = opciones_busqueda[opcion_busqueda]
                nombre_criterio, funcion_criterio = opciones_criterio[opcion_criterio]
                
                # Crear instancia y inferir red
                red_inferencia = InferirRedBooleana(funcion_busqueda, funcion_criterio, matriz_exp)
                red_inferida = red_inferencia.inferir_red()
                
                # Guardar red
                guardar_red_inferida(
                    red_inferida=red_inferida,
                    funcion_busqueda=nombre_busqueda,
                    funcion_criterio=nombre_criterio
                )
                print("\nRed inferida guardada exitosamente")
                try:
                    # Generar matriz de confusión
                    target_names = [f'Gen {i}' for i in range(num_genes)]
                    red_inferencia.generar_matriz_confusion_con_etiquetas(red_verdadera, target_names)
                except ValueError as e:
                    print(f"No se puede generar la matriz de confusión: {e}")
                except Exception as e:
                    print(f"Se produjo un error inesperado: {e}")
                # Evaluar precisión
                red_inferencia.evaluar_precision(red_verdadera)
                break
            else:
                print("Opcion no valida. Por favor, elija una opción del menú")
        
    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Se produjo un error inesperado: {e}")


--- MENÚ DE OPCIONES ---
Funciones de Búsqueda:
1. busqueda_exhaustiva
2. busqueda_sfs
3. busqueda_algoritmo_genetico

Funciones de Criterio:
1. error_absoluto
2. informacion_mutua_condicional
3. informacion_mutua_penalizada



Searching predictors with Genetic Algorithm for target gene 0
Se produjo un error inesperado: module 'pyeasyga' has no attribute 'GeneticAlgorithm'
